In [4]:
import os
from pathlib import Path
import pickle
import numpy as np
import matplotlib.pyplot as plt
import styling

In [5]:
main_path = Path.cwd().parent

In [22]:
#Collect all the data: it should be an array of dictionaries
DATA = []

##### Left Panel

In [19]:
# Logistic map data
results_path = main_path / 'logistic_map/results_training_loop_inspection/'
os.listdir(results_path)
_tmp = {}
for filename in os.listdir(results_path):
    model_name = filename.split('_')[0]
    if model_name not in _tmp:
        _tmp[model_name] = []
    with open(results_path / filename, 'rb') as f:
        restored = pickle.load(f)
    _tmp[model_name].append(restored)

LOGISTIC = {}
for model_name, model_data in _tmp.items():
    LOGISTIC[model_name] = {
        'x': np.arange(len(model_data)),
        'data': np.array(model_data).mean(axis=0)
    }

# Continuous dynamics data
with open(main_path / 'langevin.pkl', 'rb') as f:
    restored_obj = pickle.load(f)
LANGEVIN = {
    'x': restored_obj['step'],
    'data': restored_obj['eigenvalues'],
    'hlines': np.squeeze(restored_obj['reference'])
}

DATA.append([LOGISTIC, LANGEVIN])

#### Middle Panel

In [32]:
# Ordered MNIST data
reports = {}
exp_path = main_path / "ordered_MNIST/results/"
for file in os.listdir(exp_path):
    if file.endswith(".pkl"):
        with open(exp_path / file, "rb") as f:
            restored_report = pickle.load(f)
            model_name = restored_report.pop("name")
            reports[model_name] = restored_report

ORDERED_MNIST = {}
for model_name, report in reports.items():
    ORDERED_MNIST[model_name] = {
        "data": np.asanyarray(report["accuracy"]),
    }
ORDERED_MNIST["hlines"] = 0.2

# Fluid Flow data DUMMY DATA
FLUID_FLOW = {
    "DPNets": {
        "data": np.random.rand(10),
    },
    "DPNets-relaxed": {
        "data": np.random.rand(10),
    },
    "VAMPNets": {
        "data": np.random.rand(10),
    }
}

#### Right Panel

In [ ]:
from KDEpy import FFTKDE
def free_energy_surface(model_stub: str, eig_idx, bw = 'silverman'):
    lfns = report[model_stub]['left_fns']
    rfns = report[model_stub]['right_fns']
    lfns = lfns.real
    rfns = rfns.real

    eig = report[model_stub]['eigenvalues']
    lfn = lfns[:, eig_idx]
    rfn = rfns[:, eig_idx]

    #boltzmann_dist = np.where(lfns[:, 0]>0, lfns[:, 0], 1e-8)

    if lfn.ndim == 1:
        pass
        #lfn = lfn/boltzmann_dist #Normalization
    elif lfn.ndim == 2:
        pass
        #lfn = lfn/boltzmann_dist[:, None] #Normalization
    else:
        raise ValueError("Only 1 or 2 eigenvalue indices are supported")
    x, kde = FFTKDE(bw=bw).fit(lfn).evaluate(2**8) #256 bins
    fesl = -np.log(kde)
    x, kde = FFTKDE(bw=bw).fit(rfn).evaluate(2**8) #256 bins
    fesr = -np.log(kde)
    return x, fesl, fesr

free_energy_surface(model, eig_idx, bw = bw[ax_idx])